# Trabajo Práctico 1: Métricas custom para reducción de falsos positivos en clasificación binaria fraude

## Feature Engineering

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset = pd.read_csv("data/dataset_feature_engineering.csv")
dataset